In [1]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Add, Conv1D,Convolution2D, Bidirectional, LSTM, GRU, AlphaDropout, MaxPooling1D
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers
from tensorflow_addons.layers import MultiHeadAttention
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import sklearn, json
import scipy.io as io
from typing import Any, Dict
from includes.clr_callback import *

path = '/home/maria/'
dataset_path = path + 'dataset_1d/'

2022-09-02 16:29:02.663932: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
with h5py.File(dataset_path +'X_train.mat', 'r') as f:
    X_train = np.array(f['X_train']).T
print(X_train.shape)
with h5py.File(dataset_path +'X_test.mat', 'r') as f:
    X_test = np.array(f['X_test']).T
    lbl_train = io.loadmat(dataset_path +'lbl_train.mat')['lbl_train']
lbl_test = io.loadmat(dataset_path +'lbl_test.mat')['lbl_test']
print(lbl_test.shape)
print(lbl_train.shape)
Y_train = io.loadmat(dataset_path +'Y_train.mat')
Y_train = Y_train['Y_train']
print(Y_train.shape)
Y_test = io.loadmat(dataset_path +'Y_test.mat')
Y_test = Y_test['Y_test']
print(Y_test.shape)


(195500, 1024, 2)
(39100, 6)
(195500, 6)
(195500, 23)
(39100, 23)


In [3]:
classes = ['LFM', '2FSK', '4FSK', '8FSK', 'Costas', '2PSK', '4PSK', '8PSK', 'Barker', 'Huffman', 'Frank', 'P1', 'P2',
           'P3', 'P4', 'Px', 'Zadoff-Chu', 'T1', 'T2', 'T3', 'T4', 'NM', 'ruido']
I_x = X_train[:, :, 0]
Q_x = X_train[:, :, 1]
X_train[:,:,1] = np.arctan(Q_x, I_x)/ np.pi
X_train[:,:,0] = np.abs(I_x + 1j*Q_x)
I_t = X_test[:, :, 0]
Q_t = X_test[:, :, 1]
X_test[:, :, 0] = np.arctan(Q_t, I_t)/np.pi
X_test[:, :, 1] = np.abs(I_t + 1j*Q_t)
np.random.seed(2022)
X_train, Y_train, lbl_train = sklearn.utils.shuffle(X_train[:],Y_train[:], lbl_train[:], random_state=2022)
X_test, Y_test, lbl_test = sklearn.utils.shuffle(X_test[:], Y_test[:], lbl_test[:], random_state=2022)

In [17]:
class AttentionBlock(keras.Model):
    def __init__(self, name : str = 'AttentionBlock',
                 key_dim : int,
                 num_heads : int =2,
                 head_size : int = 128,
                ff_dim : int = None,
                dropout : int = 0,
                **kwargs):
        super().__init__(name=name, **kwargs)
        if ff_dim is None:
            ff_dim = head_size
        self.attention = MultiHeadAttention(num_heads=num_heads, head_size=128, dropout=dropout)
        self.attention_dropout = keras.layers.Dropout(dropout)
        self.attention_norm = keras.layers.LayerNormalization(epsilon=1e-6)
        self.conv1 = keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')
        self.dropout = keras.layers.Dropout(dropout)
        self.norm = keras.layers.LayerNormalization(epsilon=1e-6)
    
    def build(self, input_shape : int):
        self.conv2 = keras.layers.Conv1D(filters=input_shape[-1], kernel_size=1)
    
    def call(self, inputs):
        print("TRAZA")
        print(inputs)
        x = self.attention(inputs)
        print("attention")
        x = self.attention_dropout(x)
        print("attention dropout")
        x = self.attention_norm(inputs + x)
        print("attention norm")
        x = self.conv1(x)
        print("conv1")
        x = self.conv2(x)
        print("conv2")
        x = self.dropout(x)
        print("dropout")
        x = self.norm(inputs + x)
        print("norm")
        return x

In [34]:
def ModelTrunk(input_shape : int):
    X_input = tf.keras.Input(input_shape)
    num_layers = 5
    #attention_layers = [AttentionBlock(num_heads=2, head_size=128, ff_dim=None, dropout=0.1) for _ in range (num_layers)]
    #for layer in attention_layers :
      #  x = layer(x)
    attention_block = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=2)
    x = attention_block(X_input,X_input)
    x = Flatten()(x)
    x = Dense(128, activation='selu')(x)
    x = AlphaDropout(0.6)(x)
    x = Dense(128, activation='selu')(x)
    x = AlphaDropout(0.6)(x)
    x = Dense(23, activation='softmax')(x)
    model = Model(inputs = X_input, outputs = x)
    model.summary()
    return model

In [6]:
def lr_scheduler(epoch, lr, warmup_epochs=15, decay_epochs=100, initial_lr=1e-6, base_lr=1e-3, min_lr=5e-5):
    if epoch <= warmup_epochs:
        pct = epoch / warmup_epochs
        return ((base_lr - initial_lr) * pct) + initial_lr

    if epoch > warmup_epochs and epoch < warmup_epochs+decay_epochs:
        pct = 1 - ((epoch - warmup_epochs) / decay_epochs)
        return ((base_lr - min_lr) * pct) + min_lr

    return min_lr

callbacks = [keras.callbacks.LearningRateScheduler(schedule=lr_scheduler, verbose=0)]

In [35]:
model = ModelTrunk(X_train.shape[1:])


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 1024, 2)]    0                                            
__________________________________________________________________________________________________
multi_head_attention_15 (MultiH (None, 1024, 2)      46          input_14[0][0]                   
                                                                 input_14[0][0]                   
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 2048)         0           multi_head_attention_15[0][0]    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 128)          262272      flatten_1[0][0]              

In [36]:
model.compile(optimizer=optimizers.Adam(1e-7), loss='categorical_crossentropy', metrics=['accuracy'])


In [38]:
output_path = path + 'Results/model_stft'
clr_triangular = CyclicLR(mode='triangular', base_lr=1e-7, max_lr=1e-4, step_size= 4 * (X_train.shape[0] // 256))
c=[clr_triangular,ModelCheckpoint(filepath= output_path +'best_model.h5', monitor='val_loss', save_best_only=True)]


In [ ]:
history = model.fit(X_train, Y_train, epochs = 50, batch_size = 256, callbacks = c, validation_data=(X_test, Y_test))
with open(output_path +'history_rnn.json', 'w') as f:
    json.dump(history.history, f)
model_json = model.to_json()
with open(output_path +'model_rnn.json', "w") as json_file:
    json_file.write(model_json)

In [20]:
input_tensor = tf.keras.Input(shape=[1024,2])

In [29]:
layer = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=2)
target = Input(shape=[1024,2])
output_tensor = layer(target,target)

In [16]:
X_train.shape[1:]

(1024, 2)